# Data Exploration

Dataset used in this project is small "Sparkify" dataset containing different users interactions with the music streaming service (like Spotify or Pandora). Interactions could be playing the song, loginig out, giving the thumbs up etc. 

In this project due to technical limitations I'm going to work on a tiny subset (128MB) of the full dataset available (12GB).

My objective is to create a machine learning model for churn prediction, but before I do that I need to get familiar with the dataset.

In [1]:
# import libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
import pandas as pd

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify_Project") \
    .getOrCreate()

In [3]:
# loading dataset
path = "mini_sparkify_event_data.json"
data = spark.read.json(path)

In [4]:
# Let's look at the schema first
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# Let's look at some example records
data.show(5, False)

+----------------+---------+---------+------+-------------+--------+---------+-----+------------------------------+------+--------+-------------+---------+-----------------------------+------+-------------+---------------------------------------------------------------------------------------------------------------+------+
|artist          |auth     |firstName|gender|itemInSession|lastName|length   |level|location                      |method|page    |registration |sessionId|song                         |status|ts           |userAgent                                                                                                      |userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+------------------------------+------+--------+-------------+---------+-----------------------------+------+-------------+---------------------------------------------------------------------------------------------------------------+------+
|Martha Tilston  |Logg

In [6]:
# What is number of records in this dataset:
data.count()

286500

In [7]:
# What is the number of unique users:
data.select("userId").dropDuplicates().count()

226

In [8]:
# Let's see what interactions are included in the dataset, this information is kept in "page" column
data.groupBy("page").count().orderBy(F.desc("count")).show()

+--------------------+------+
|                page| count|
+--------------------+------+
|            NextSong|228108|
|                Home| 14457|
|           Thumbs Up| 12551|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|         Roll Advert|  3933|
|               Login|  3241|
|              Logout|  3226|
|         Thumbs Down|  2546|
|           Downgrade|  2055|
|                Help|  1726|
|            Settings|  1514|
|               About|   924|
|             Upgrade|   499|
|       Save Settings|   310|
|               Error|   258|
|      Submit Upgrade|   159|
|    Submit Downgrade|    63|
|              Cancel|    52|
|Cancellation Conf...|    52|
+--------------------+------+
only showing top 20 rows



In [9]:
# Let's take closer look into userId and sessionId columns:
data.describe("userId", "sessionId").show()

+-------+------------------+-----------------+
|summary|            userId|        sessionId|
+-------+------------------+-----------------+
|  count|            286500|           286500|
|   mean| 59682.02278593872|1041.526554973822|
| stddev|109091.94999910535|726.7762634630799|
|    min|                  |                1|
|    max|                99|             2474|
+-------+------------------+-----------------+



In [10]:
# It seems there are some ampty values for userId

In [11]:
# Let's also take a look on ts column:
data.select("userId", "ts").orderBy("userId", "ts").show(20, False)

+------+-------------+
|userId|ts           |
+------+-------------+
|      |1538355745000|
|      |1538355807000|
|      |1538355841000|
|      |1538355842000|
|      |1538356678000|
|      |1538356679000|
|      |1538358102000|
|      |1538360117000|
|      |1538361527000|
|      |1538361528000|
|      |1538362007000|
|      |1538362095000|
|      |1538362096000|
|      |1538363488000|
|      |1538363494000|
|      |1538363503000|
|      |1538364254000|
|      |1538364255000|
|      |1538364750000|
|      |1538370681000|
+------+-------------+
only showing top 20 rows



# Data Cleaning

In this step I'm gonna get rid of empty values for userId and sessionId and change format of "ts" column

In [12]:
data_valid = (
    data.
    dropna(how = "any", subset = ["userId", "sessionId"]).
    filter(data["userId"] != "").
    withColumn("ts", F.from_unixtime(data.ts / 1000.0, "yyyy-MM-dd HH:mm:ss"))
)

In [13]:
# Let's check for empty values again:
data_valid.describe("userId", "sessionId").show()

+-------+------------------+------------------+
|summary|            userId|         sessionId|
+-------+------------------+------------------+
|  count|            278154|            278154|
|   mean| 59682.02278593872|1042.5616241362698|
| stddev|109091.94999910535| 726.5010362219824|
|    min|                10|                 1|
|    max|                99|              2474|
+-------+------------------+------------------+



In [14]:
# Let's check number of users again
data_valid.select("userId").dropDuplicates().count()

225

In [15]:
# We have one user less

In [16]:
#Let's look at timestamp column:
data_valid.select("ts").show(20, False)

+-------------------+
|ts                 |
+-------------------+
|2018-10-01 02:01:57|
|2018-10-01 02:03:00|
|2018-10-01 02:06:34|
|2018-10-01 02:06:56|
|2018-10-01 02:11:16|
|2018-10-01 02:11:18|
|2018-10-01 02:14:46|
|2018-10-01 02:14:59|
|2018-10-01 02:15:05|
|2018-10-01 02:18:04|
|2018-10-01 02:19:06|
|2018-10-01 02:19:10|
|2018-10-01 02:20:18|
|2018-10-01 02:22:55|
|2018-10-01 02:22:56|
|2018-10-01 02:24:01|
|2018-10-01 02:26:16|
|2018-10-01 02:27:48|
|2018-10-01 02:28:07|
|2018-10-01 02:29:04|
+-------------------+
only showing top 20 rows



In [17]:
# It looks much better

In [20]:
# Let's save our cleaned dataset
path_save = "mini_sparkify_event_data_cleaned.json"
data_valid.coalesce(1).write.mode("overwrite").format('json').save(path_save)